In [ ]:
import pandas as pd
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view
import os, pickle
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

from itertools import product

import sys, os

import trajectory as T                      # trajectory generation
import optimizer as O                       # stochastic gradient descent optimizer
import solver as S                          # MDP solver (value-iteration)
import plot as P


num_data = 355504


np.random.seed(66)

def to_interval(istr):
    c_left = istr[0]=='['
    c_right = istr[-1]==']'
    closed = {(True, False): 'left',
              (False, True): 'right',
              (True, True): 'both',
              (False, False): 'neither'
              }[c_left, c_right]
    left, right = map(pd.to_datetime, istr[1:-1].split(','))
    return pd.Interval(left, right, closed)

re_split = False
frac = [0.4,0.2,0.4]
assert np.sum(frac) == 1
frac = np.cumsum(frac)
print (frac)
data_save_path= 'data/'

def sliding(gs, window_size = 6):
    npr_l = []
    for g in gs:
        npr = np.concatenate([np.zeros([window_size-1, g.shape[1]]),g])
        npr_l.append(sliding_window_view(npr, (window_size, g.shape[1])).squeeze(1))
    return np.vstack(npr_l)

[0.4 0.6 1. ]


# LOADING THE DATA

In [ ]:
# if re_split:


aggr_df = pd.read_csv('mimic_iv_hypotensive_cut2.csv',sep = ',', header = 0,converters={1:to_interval}).set_index(['stay_id','time']).sort_index()
# create action bins (four actions in total)
aggr_df['action'] = aggr_df['bolus(binary)']*2 + aggr_df['vaso(binary)']
all_idx = np.random.permutation(aggr_df.index.get_level_values(0).unique())
train_df = aggr_df.loc[all_idx[:int(len(all_idx)*frac[0])]].sort_index()
test_df = aggr_df.loc[all_idx[int(len(all_idx)*frac[0]):int(len(all_idx)*frac[1])]].sort_index()
valid_df = aggr_df.loc[all_idx[int(len(all_idx)*frac[1]):]].sort_index()
# print (np.unique(train_df['action'],return_counts=True)[1]*1./len(train_df))
# pickle.dump([train_df, test_df, valid_df], open(data_save_path+'processed_mimic_hyp_2.pkl','wb'))
drop_columns = ['vaso(amount)','bolus(amount)',\
            'any_treatment(binary)','vaso(binary)','bolus(binary)']




# Adjusting Cluster Assignments


In [ ]:
# load cluster information
aggregated_cluster_df = pd.read_csv('diff_cluster_algos.csv')

print(aggregated_cluster_df.shape)

(355504, 19)


In [ ]:
# view loaded data
# 'k_cluster', 'm_cluster', 'b_cluster', 'h_cluster'
print("columns:", aggregated_cluster_df.columns)
aggregated_cluster_df.head()

columns: Index(['creatinine', 'fraction_inspired_oxygen', 'lactate', 'urine_output',
       'alanine_aminotransferase', 'asparate_aminotransferase',
       'mean_blood_pressure', 'diastolic_blood_pressure',
       'systolic_blood_pressure', 'gcs', 'partial_pressure_of_oxygen',
       'heart_rate', 'temperature', 'respiratory_rate', 'action', 'k_cluster',
       'm_cluster', 'b_cluster', 'h_cluster'],
      dtype='object')


,creatinine,fraction_inspired_oxygen,lactate,urine_output,alanine_aminotransferase,asparate_aminotransferase,mean_blood_pressure,diastolic_blood_pressure,systolic_blood_pressure,gcs,partial_pressure_of_oxygen,heart_rate,temperature,respiratory_rate,action,k_cluster,m_cluster,b_cluster,h_cluster
0,-0.422008,-1.760743,-0.182521,-0.225783,-0.288689,-0.265706,0.404836,0.391566,0.36885,-2.479374,-0.186486,86.0,37.0,19.0,0.0,38,86,53,-1
1,-0.422008,-1.760743,-0.182521,-0.225783,-0.288689,-0.265706,0.404836,0.391566,0.36885,-2.479374,-0.186486,86.0,37.0,19.0,0.0,38,86,53,-1
2,-0.422008,-1.760743,0.360532,-0.225783,-0.288689,-0.265706,0.404836,0.391566,0.36885,-2.479374,2.356989,86.0,37.0,19.0,0.0,38,43,53,-1
3,-0.422008,-1.760743,0.360532,-0.225783,-0.288689,-0.265706,0.404836,0.391566,0.36885,-2.479374,2.356989,86.0,37.0,19.0,0.0,38,43,53,-1
4,-0.422008,-1.760743,0.360532,-0.225783,-0.288689,-0.265706,0.404836,0.391566,0.36885,-2.479374,2.356989,86.0,37.0,19.0,0.0,38,43,53,-1


In [ ]:
# change this value for each cluster assignment and run the rest of the notebook
cluster_algo = 'm_cluster'
cluster_name = 'minibatchK'

print("before: ", aggregated_cluster_df.shape)
aggregated_cluster_df.dropna(subset=[cluster_algo], inplace=True)
print("after: ", aggregated_cluster_df.shape)


CLUSTER_LIST= aggregated_cluster_df[cluster_algo]


# variables to change
smoothing_value = 1
discount_val = 0.9

before:  (355504, 19)
after:  (355504, 19)


In [ ]:
num_clusters = 100

# Data Cleaning


In [ ]:
# for now drop indicators about bolus and vaso
train_df = train_df.drop(columns=drop_columns)
test_df = test_df.drop(columns=drop_columns)
valid_df = valid_df.drop(columns=drop_columns)

#### imputation
impute_table = pd.read_csv('mimic_iv_hypotensive_cut2_impute_table.csv',sep=',',header=0).set_index(['feature'])
train_df = train_df.fillna(method='ffill')
test_df = test_df.fillna(method='ffill')
valid_df = valid_df.fillna(method='ffill')




for f in impute_table.index:
    train_df[f] = train_df[f].fillna(value = impute_table.loc[f].values[0])
    test_df[f] = test_df[f].fillna(value = impute_table.loc[f].values[0])
    valid_df[f] = valid_df[f].fillna(value = impute_table.loc[f].values[0])


data_non_normalized_df = pd.concat([train_df, valid_df, test_df], axis=0, ignore_index=False).head(num_data).copy()


#### standard normalization ####
normalize_features = ['creatinine', 'fraction_inspired_oxygen', 'lactate', 'urine_output',
                  'alanine_aminotransferase', 'asparate_aminotransferase',
                  'mean_blood_pressure', 'diastolic_blood_pressure',
                  'systolic_blood_pressure', 'gcs', 'partial_pressure_of_oxygen']
mu, std = (train_df[normalize_features]).mean().values,(train_df[normalize_features]).std().values
train_df[normalize_features] = (train_df[normalize_features] - mu)/std
test_df[normalize_features] = (test_df[normalize_features] - mu)/std
valid_df[normalize_features] = (valid_df[normalize_features] - mu)/std




### create data matrix ####
X_train = train_df.loc[:,train_df.columns!='action']
y_train = train_df['action']

X_test = test_df.loc[:,test_df.columns!='action']
y_test = test_df['action']

X_valid = valid_df.loc[:, valid_df.columns!='action']
y_valid = valid_df['action']

In [ ]:
X_df = pd.concat([X_train, X_valid, X_test], axis=0, ignore_index=True).copy()
y_df = pd.concat([y_train, y_valid, y_test], axis=0, ignore_index=True).copy()


In [ ]:
data_df = pd.concat([train_df, valid_df, test_df], axis=0, ignore_index=False).copy()
# data_df = data_df.head(num_data).copy()
# X_df = X_df.head(num_data).copy()
# y_df = y_df.head(num_data).copy()
print(len(data_df))
print(len(X_df))
print(len(y_df))
print(len(data_non_normalized_df))

print("columns for data_df:")
print(list(data_df.columns))
print("columns for X_df:")
print(list(X_df.columns))

355504
355504
355504
355504
columns for data_df:
['creatinine', 'fraction_inspired_oxygen', 'lactate', 'urine_output', 'alanine_aminotransferase', 'asparate_aminotransferase', 'mean_blood_pressure', 'diastolic_blood_pressure', 'systolic_blood_pressure', 'gcs', 'partial_pressure_of_oxygen', 'heart_rate', 'temperature', 'respiratory_rate', 'action']
columns for X_df:
['creatinine', 'fraction_inspired_oxygen', 'lactate', 'urine_output', 'alanine_aminotransferase', 'asparate_aminotransferase', 'mean_blood_pressure', 'diastolic_blood_pressure', 'systolic_blood_pressure', 'gcs', 'partial_pressure_of_oxygen', 'heart_rate', 'temperature', 'respiratory_rate']


# Normalized version of the data

In [ ]:
data_df.head()

creatinine  \
stay_id  time                                                     
30004811 [2139-10-06 10:40:29, 2139-10-06 11:40:29)   -0.422008   
         [2139-10-06 11:40:29, 2139-10-06 12:40:29)   -0.422008   
         [2139-10-06 12:40:29, 2139-10-06 13:40:29)   -0.422008   
         [2139-10-06 13:40:29, 2139-10-06 14:40:29)   -0.422008   
         [2139-10-06 14:40:29, 2139-10-06 15:40:29)   -0.422008   

                                                     fraction_inspired_oxygen  \
stay_id  time                                                                   
30004811 [2139-10-06 10:40:29, 2139-10-06 11:40:29)                 -1.760743   
         [2139-10-06 11:40:29, 2139-10-06 12:40:29)                 -1.760743   
         [2139-10-06 12:40:29, 2139-10-06 13:40:29)                 -1.760743   
         [2139-10-06 13:40:29, 2139-10-06 14:40:29)                 -1.760743   
         [2139-10-06 14:40:29, 2139-10-06 15:40:29)                 -1.760743   

                                                      lactate  urine_output  \
stay_id  time                                                                 
30004811 [2139-10-06 10:40:29, 2139-10-06 11:40:29) -0.182521     -0.225783   
         [2139-10-06 11:40:29, 2139-10-06 12:40:29) -0.182521     -0.225783   
         [2139-10-06 12:40:29, 2139-10-06 13:40:29)  0.360532     -0.225783   
         [2139-10-06 13:40:29, 2139-10-06 14:40:29)  0.360532     -0.225783   
         [2139-10-06 14:40:29, 2139-10-06 15:40:29)  0.360532     -0.225783   

                                                     alanine_aminotransferase  \
stay_id  time                                                                   
30004811 [2139-10-06 10:40:29, 2139-10-06 11:40:29)                 -0.288689   
         [2139-10-06 11:40:29, 2139-10-06 12:40:29)                 -0.288689   
         [2139-10-06 12:40:29, 2139-10-06 13:40:29)                 -0.288689   
         [2139-10-06 13:40:29, 2139-10-06 14:40:29)                 -0.288689   
         [2139-10-06 14:40:29, 2139-10-06 15:40:29)                 -0.288689   

                                                     asparate_aminotransferase  \
stay_id  time                                                                    
30004811 [2139-10-06 10:40:29, 2139-10-06 11:40:29)                  -0.265706   
         [2139-10-06 11:40:29, 2139-10-06 12:40:29)                  -0.265706   
         [2139-10-06 12:40:29, 2139-10-06 13:40:29)                  -0.265706   
         [2139-10-06 13:40:29, 2139-10-06 14:40:29)                  -0.265706   
         [2139-10-06 14:40:29, 2139-10-06 15:40:29)                  -0.265706   

                                                     mean_blood_pressure  \
stay_id  time                                                              
30004811 [2139-10-06 10:40:29, 2139-10-06 11:40:29)             0.404836   
         [2139-10-06 11:40:29, 2139-10-06 12:40:29)             0.404836   
         [2139-10-06 12:40:29, 2139-10-06 13:40:29)             0.404836   
         [2139-10-06 13:40:29, 2139-10-06 14:40:29)             0.404836   
         [2139-10-06 14:40:29, 2139-10-06 15:40:29)             0.404836   

                                                     diastolic_blood_pressure  \
stay_id  time                                                                   
30004811 [2139-10-06 10:40:29, 2139-10-06 11:40:29)                  0.391566   
         [2139-10-06 11:40:29, 2139-10-06 12:40:29)                  0.391566   
         [2139-10-06 12:40:29, 2139-10-06 13:40:29)                  0.391566   
         [2139-10-06 13:40:29, 2139-10-06 14:40:29)                  0.391566   
         [2139-10-06 14:40:29, 2139-10-06 15:40:29)                  0.391566   

                                                     systolic_blood_pressure  \
stay_id  time                                                                  
30004811 [2139-10-06 10:40:29, 2139-10-06 11:40:29)      

# Unormalized version of the data

In [ ]:
data_non_normalized_df.head()

creatinine  \
stay_id  time                                                     
30004811 [2139-10-06 10:40:29, 2139-10-06 11:40:29)         1.0   
         [2139-10-06 11:40:29, 2139-10-06 12:40:29)         1.0   
         [2139-10-06 12:40:29, 2139-10-06 13:40:29)         1.0   
         [2139-10-06 13:40:29, 2139-10-06 14:40:29)         1.0   
         [2139-10-06 14:40:29, 2139-10-06 15:40:29)         1.0   

                                                     fraction_inspired_oxygen  \
stay_id  time                                                                   
30004811 [2139-10-06 10:40:29, 2139-10-06 11:40:29)                      0.21   
         [2139-10-06 11:40:29, 2139-10-06 12:40:29)                      0.21   
         [2139-10-06 12:40:29, 2139-10-06 13:40:29)                      0.21   
         [2139-10-06 13:40:29, 2139-10-06 14:40:29)                      0.21   
         [2139-10-06 14:40:29, 2139-10-06 15:40:29)                      0.21   

                                                     lactate  urine_output  \
stay_id  time                                                                
30004811 [2139-10-06 10:40:29, 2139-10-06 11:40:29)      1.8          80.0   
         [2139-10-06 11:40:29, 2139-10-06 12:40:29)      1.8          80.0   
         [2139-10-06 12:40:29, 2139-10-06 13:40:29)      3.0          80.0   
         [2139-10-06 13:40:29, 2139-10-06 14:40:29)      3.0          80.0   
         [2139-10-06 14:40:29, 2139-10-06 15:40:29)      3.0          80.0   

                                                     alanine_aminotransferase  \
stay_id  time                                                                   
30004811 [2139-10-06 10:40:29, 2139-10-06 11:40:29)                      34.0   
         [2139-10-06 11:40:29, 2139-10-06 12:40:29)                      34.0   
         [2139-10-06 12:40:29, 2139-10-06 13:40:29)                      34.0   
         [2139-10-06 13:40:29, 2139-10-06 14:40:29)                      34.0   
         [2139-10-06 14:40:29, 2139-10-06 15:40:29)                      34.0   

                                                     asparate_aminotransferase  \
stay_id  time                                                                    
30004811 [2139-10-06 10:40:29, 2139-10-06 11:40:29)                       40.0   
         [2139-10-06 11:40:29, 2139-10-06 12:40:29)                       40.0   
         [2139-10-06 12:40:29, 2139-10-06 13:40:29)                       40.0   
         [2139-10-06 13:40:29, 2139-10-06 14:40:29)                       40.0   
         [2139-10-06 14:40:29, 2139-10-06 15:40:29)                       40.0   

                                                     mean_blood_pressure  \
stay_id  time                                                              
30004811 [2139-10-06 10:40:29, 2139-10-06 11:40:29)                 77.0   
         [2139-10-06 11:40:29, 2139-10-06 12:40:29)                 77.0   
         [2139-10-06 12:40:29, 2139-10-06 13:40:29)                 77.0   
         [2139-10-06 13:40:29, 2139-10-06 14:40:29)                 77.0   
         [2139-10-06 14:40:29, 2139-10-06 15:40:29)                 77.0   

                                                     diastolic_blood_pressure  \
stay_id  time                                                                   
30004811 [2139-10-06 10:40:29, 2139-10-06 11:40:29)                      59.0   
         [2139-10-06 11:40:29, 2139-10-06 12:40:29)                      59.0   
         [2139-10-06 12:40:29, 2139-10-06 13:40:29)                      59.0   
         [2139-10-06 13:40:29, 2139-10-06 14:40:29)                      59.0   
         [2139-10-06 14:40:29, 2139-10-06 15:40:29)                      59.0   

                                                     systolic_blood_pressure  \
stay_id  time                                                                  
30004811 [2139-10-06 10:40:29, 2139-10-06 11:40:29)             

# Matrix form of the data (Normalized and features only)

In [ ]:
X_df.head()

,creatinine,fraction_inspired_oxygen,lactate,urine_output,alanine_aminotransferase,asparate_aminotransferase,mean_blood_pressure,diastolic_blood_pressure,systolic_blood_pressure,gcs,partial_pressure_of_oxygen,heart_rate,temperature,respiratory_rate
0,-0.422008,-1.760743,-0.182521,-0.225783,-0.288689,-0.265706,0.404836,0.391566,0.36885,-2.479374,-0.186486,86.0,37.0,19.0
1,-0.422008,-1.760743,-0.182521,-0.225783,-0.288689,-0.265706,0.404836,0.391566,0.36885,-2.479374,-0.186486,86.0,37.0,19.0
2,-0.422008,-1.760743,0.360532,-0.225783,-0.288689,-0.265706,0.404836,0.391566,0.36885,-2.479374,2.356989,86.0,37.0,19.0
3,-0.422008,-1.760743,0.360532,-0.225783,-0.288689,-0.265706,0.404836,0.391566,0.36885,-2.479374,2.356989,86.0,37.0,19.0
4,-0.422008,-1.760743,0.360532,-0.225783,-0.288689,-0.265706,0.404836,0.391566,0.36885,-2.479374,2.356989,86.0,37.0,19.0


# Corresponding output data for training BC (corresponding treatments for each data point in X_df)

In [ ]:
y_df.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: action, dtype: float64

# Clustering the feature space to extract a discrete state space form the clusters

In [ ]:
# Looking at the values counts for each cluster

np.unique(CLUSTER_LIST, return_counts = True)[1]

array([ 5357,  1704,  8051,  3280,  3268,  3457,  6136,  1106,  3580,
        1599,  4779,  3200,  1938,  5733,  1386,  2207,  4911,  6024,
        6370,  2216,  4558,  2129,  7200,  6549,  5249,  5132,  3931,
        3434,  1567,  2975,  6631,  4580,  2092,  5221,  3040,  3011,
        6804,  4614,  3387,   912,  1786,   177,  3864,  8043,   890,
         858,   614,  1865,   505,  1064,  7543,  8026,  1540,  8750,
        3712,  1587,   418, 10237,  5876,  3643,  5175,  2327,  3503,
        1734,  6336,   796,   399,  1141,  4741,  4701,  3257,  8165,
        2869,  3684,  1932,  5086,  3164,  4215,  6915,  6223,  2830,
         711,  1509,  3362,   854,  2133,  3028,  4535,  5149,  3445,
         753,  4802,  2218,  3114,  2468,   432,  1265,  2731,   476,
        3010])

In [ ]:
print(data_non_normalized_df.shape, len(CLUSTER_LIST))
print(aggregated_cluster_df.shape)
print(type(np.array(CLUSTER_LIST)))

np_CLUSTER_LIST = np.array(CLUSTER_LIST)

(355504, 15) 355504
(355504, 19)
<class 'numpy.ndarray'>


In [ ]:
# Assigning each data point to a cluster

X_df['cluster'] = np_CLUSTER_LIST.copy()
data_df['cluster'] = np_CLUSTER_LIST.copy()
data_non_normalized_df['cluster'] = np_CLUSTER_LIST.copy()
data_non_normalized_df.head()

creatinine  \
stay_id  time                                                     
30004811 [2139-10-06 10:40:29, 2139-10-06 11:40:29)         1.0   
         [2139-10-06 11:40:29, 2139-10-06 12:40:29)         1.0   
         [2139-10-06 12:40:29, 2139-10-06 13:40:29)         1.0   
         [2139-10-06 13:40:29, 2139-10-06 14:40:29)         1.0   
         [2139-10-06 14:40:29, 2139-10-06 15:40:29)         1.0   

                                                     fraction_inspired_oxygen  \
stay_id  time                                                                   
30004811 [2139-10-06 10:40:29, 2139-10-06 11:40:29)                      0.21   
         [2139-10-06 11:40:29, 2139-10-06 12:40:29)                      0.21   
         [2139-10-06 12:40:29, 2139-10-06 13:40:29)                      0.21   
         [2139-10-06 13:40:29, 2139-10-06 14:40:29)                      0.21   
         [2139-10-06 14:40:29, 2139-10-06 15:40:29)                      0.21   

                                                     lactate  urine_output  \
stay_id  time                                                                
30004811 [2139-10-06 10:40:29, 2139-10-06 11:40:29)      1.8          80.0   
         [2139-10-06 11:40:29, 2139-10-06 12:40:29)      1.8          80.0   
         [2139-10-06 12:40:29, 2139-10-06 13:40:29)      3.0          80.0   
         [2139-10-06 13:40:29, 2139-10-06 14:40:29)      3.0          80.0   
         [2139-10-06 14:40:29, 2139-10-06 15:40:29)      3.0          80.0   

                                                     alanine_aminotransferase  \
stay_id  time                                                                   
30004811 [2139-10-06 10:40:29, 2139-10-06 11:40:29)                      34.0   
         [2139-10-06 11:40:29, 2139-10-06 12:40:29)                      34.0   
         [2139-10-06 12:40:29, 2139-10-06 13:40:29)                      34.0   
         [2139-10-06 13:40:29, 2139-10-06 14:40:29)                      34.0   
         [2139-10-06 14:40:29, 2139-10-06 15:40:29)                      34.0   

                                                     asparate_aminotransferase  \
stay_id  time                                                                    
30004811 [2139-10-06 10:40:29, 2139-10-06 11:40:29)                       40.0   
         [2139-10-06 11:40:29, 2139-10-06 12:40:29)                       40.0   
         [2139-10-06 12:40:29, 2139-10-06 13:40:29)                       40.0   
         [2139-10-06 13:40:29, 2139-10-06 14:40:29)                       40.0   
         [2139-10-06 14:40:29, 2139-10-06 15:40:29)                       40.0   

                                                     mean_blood_pressure  \
stay_id  time                                                              
30004811 [2139-10-06 10:40:29, 2139-10-06 11:40:29)                 77.0   
         [2139-10-06 11:40:29, 2139-10-06 12:40:29)                 77.0   
         [2139-10-06 12:40:29, 2139-10-06 13:40:29)                 77.0   
         [2139-10-06 13:40:29, 2139-10-06 14:40:29)                 77.0   
         [2139-10-06 14:40:29, 2139-10-06 15:40:29)                 77.0   

                                                     diastolic_blood_pressure  \
stay_id  time                                                                   
30004811 [2139-10-06 10:40:29, 2139-10-06 11:40:29)                      59.0   
         [2139-10-06 11:40:29, 2139-10-06 12:40:29)                      59.0   
         [2139-10-06 12:40:29, 2139-10-06 13:40:29)                      59.0   
         [2139-10-06 13:40:29, 2139-10-06 14:40:29)                      59.0   
         [2139-10-06 14:40:29, 2139-10-06 15:40:29)                      59.0   

                                                     systolic_blood_pressure  \
stay_id  time                                                                  
30004811 [2139-10-06 10:40:29, 2139-10-06 11:40:29)             

In [ ]:
X_df.head()

,creatinine,fraction_inspired_oxygen,lactate,urine_output,alanine_aminotransferase,asparate_aminotransferase,mean_blood_pressure,diastolic_blood_pressure,systolic_blood_pressure,gcs,partial_pressure_of_oxygen,heart_rate,temperature,respiratory_rate,cluster
0,-0.422008,-1.760743,-0.182521,-0.225783,-0.288689,-0.265706,0.404836,0.391566,0.36885,-2.479374,-0.186486,86.0,37.0,19.0,86
1,-0.422008,-1.760743,-0.182521,-0.225783,-0.288689,-0.265706,0.404836,0.391566,0.36885,-2.479374,-0.186486,86.0,37.0,19.0,86
2,-0.422008,-1.760743,0.360532,-0.225783,-0.288689,-0.265706,0.404836,0.391566,0.36885,-2.479374,2.356989,86.0,37.0,19.0,43
3,-0.422008,-1.760743,0.360532,-0.225783,-0.288689,-0.265706,0.404836,0.391566,0.36885,-2.479374,2.356989,86.0,37.0,19.0,43
4,-0.422008,-1.760743,0.360532,-0.225783,-0.288689,-0.265706,0.404836,0.391566,0.36885,-2.479374,2.356989,86.0,37.0,19.0,43


# Converting the data into trajectories to input to an IRL algorithm Note this is the same format of trajectories we used for HW1 and HW2.

In [ ]:
unique_stay_ids = data_df.index.get_level_values('stay_id').unique()

trajectories = []


for stay_id in unique_stay_ids:


  states, actions = data_df.loc[stay_id]['cluster'], data_df.loc[stay_id]['action']

  trajectory = []
  for i in range(len(states) - 1):
    trajectory.append((states[i], int(actions[i]), states[i+1] ))

  trajectories.append(T.Trajectory(trajectory))

We need to store all possible terminal states from the trajectories list. (Needed to calculate the normalizing constant in MaxEnt)

In [ ]:
terminal_states = []

for traj in trajectories:
  terminal_states.append(traj._t[-1][-1])

terminal_states = list(set(terminal_states))

In [ ]:
terminal_states

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99]

# Distribution of the treatments given in our data. (Most of the time no treatment is given, might vary on depending on how you cluster the data)

In [ ]:
y_df.value_counts()

0.0    195786
1.0    135305
3.0     15978
2.0      8435
Name: action, dtype: int64

# BC Policy example

In [ ]:
# # Convert states and actions to one-hot encoding
state_encoder = OneHotEncoder(sparse=False, categories= [np.arange(num_clusters)])
action_encoder = OneHotEncoder(sparse=False, categories= [np.arange(4)])


states_onehot = state_encoder.fit_transform(X_df['cluster'].to_numpy().reshape(-1, 1))
actions_onehot = action_encoder.fit_transform(y_df.to_numpy().reshape(-1, 1))


# # # Define neural network architecture
# model = tf.keras.Sequential([
#     tf.keras.layers.Dense(32, activation='relu', input_shape=(states_onehot.shape[1],)),
#     tf.keras.layers.Dense(64, activation='relu'),
#     tf.keras.layers.Dense(128, activation='relu'),
#     tf.keras.layers.Dense(64, activation='relu'),
#     tf.keras.layers.Dense(32, activation='relu'),
#     tf.keras.layers.Dense(actions_onehot.shape[1], activation='softmax')  # Output layer with softmax for discrete actions
# ])

# # # Compile the model
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics= ['accuracy'])

# # # Train the model
# model.fit(states_onehot, actions_onehot,  epochs=5, batch_size=128)

# # # Evaluate the model
# test_loss = model.evaluate(states_onehot, actions_onehot)
# print("Test Loss:", test_loss)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
# bc_policy = np.argmax(model.predict(state_encoder.transform(np.arange(num_clusters).reshape(-1, 1))), axis =1)
# bc_policy

In [ ]:
# fig, axes = plt.subplots(nrows=20, ncols=5, figsize=(35, 45))


# for i, ax in enumerate(axes.flatten()):
#     data_df[data_df['cluster'] == i]['action'].value_counts().plot(kind = 'bar', ax = ax, title = 'Cluster: ' + str(i+ 1))
#     ax.set_ylabel('Counts')

# Estimating the Transition Dynamics using the MLE (feel free to play around with the smoothing_value)

In [ ]:
p_transition = np.zeros((num_clusters, num_clusters, 4)) + smoothing_value

for traj in trajectories:
  for tran in traj._t:
    p_transition[tran[0], tran[2], tran[1]] +=1

p_transition = p_transition/ p_transition.sum(axis = 1)[:, np.newaxis, :]

In [ ]:
p_transition

array([[[2.73511648e-01, 2.47170935e-01, 1.79856115e-01, 9.66386555e-02],
        [8.62812770e-04, 5.95592615e-04, 3.59712230e-03, 4.20168067e-03],
        [8.34052344e-03, 7.14711138e-03, 1.43884892e-02, 8.40336134e-03],
        ...,
        [2.87604257e-04, 5.95592615e-04, 3.59712230e-03, 4.20168067e-03],
        [2.87604257e-04, 5.95592615e-04, 3.59712230e-03, 4.20168067e-03],
        [5.75208513e-04, 1.78677784e-03, 3.59712230e-03, 4.20168067e-03]],

       [[2.13675214e-03, 2.53807107e-03, 8.47457627e-03, 7.57575758e-03],
        [1.56695157e-01, 8.88324873e-02, 8.47457627e-03, 1.51515152e-02],
        [8.54700855e-03, 7.61421320e-03, 8.47457627e-03, 7.57575758e-03],
        ...,
        [1.42450142e-03, 2.53807107e-03, 8.47457627e-03, 7.57575758e-03],
        [3.56125356e-03, 5.07614213e-03, 8.47457627e-03, 7.57575758e-03],
        [6.41025641e-03, 1.52284264e-02, 8.47457627e-03, 7.57575758e-03]],

       [[6.45855759e-03, 3.78266850e-03, 1.83150183e-02, 6.12244898e-03],
        

# Max Causal Entropy

# Feel free to play with the discount factor

In [ ]:
"""
Maximum Entropy Inverse Reinforcement Learning and Maximum Causal Entropy
Inverse Reinforcement Learning.

Based on the corresponding paper by B. Ziebart et al. (2008) and the Thesis
by Ziebart (2010).
"""

import numpy as np
from itertools import product


# -- common functions ----------------------------------------------------------

def feature_expectation_from_trajectories(features, trajectories):
    """
    Compute the feature expectation of the given trajectories.

    Simply counts the number of visitations to each feature-instance and
    divides them by the number of trajectories.

    Args:
        features: The feature-matrix (e.g. as numpy array), mapping states
            to features, i.e. a matrix of shape (n_states x n_features).
        trajectories: A list or iterator of `Trajectory` instances.

    Returns:
        The feature-expectation of the provided trajectories as map
        `[state: Integer] -> feature_expectation: Float`.
    """
    n_states, n_features = features.shape

    fe = np.zeros(n_features)

    for t in trajectories:
        for s in t.states():
            fe += features[s, :]

    return fe / len(trajectories)


def initial_probabilities_from_trajectories(n_states, trajectories):
    """
    Compute the probability of a state being a starting state using the
    given trajectories.

    Args:
        n_states: The number of states.
        trajectories: A list or iterator of `Trajectory` instances.

    Returns:
        The probability of a state being a starting-state as map
        `[state: Integer] -> probability: Float`.
    """
    p = np.zeros(n_states)

    for t in trajectories:
        p[t.transitions()[0][0]] += 1.0

    return p / len(trajectories)


def expected_svf_from_policy(p_transition, p_initial, terminal, p_action, eps=1e-5):
    """
    Compute the expected state visitation frequency using the given local
    action probabilities.

    This is the forward pass of Algorithm 1 of the Maximum Entropy IRL paper
    by Ziebart et al. (2008). Alternatively, it can also be found as
    Algorithm 9.3 in in Ziebart's thesis (2010).

    It has been slightly adapted for convergence, by forcing transition
    probabilities from terminal stats to be zero.

    Args:
        p_transition: The transition probabilities of the MDP as table
            `[from: Integer, to: Integer, action: Integer] -> probability: Float`
            specifying the probability of a transition from state `from` to
            state `to` via action `action` to succeed.
        p_initial: The probability of a state being an initial state as map
            `[state: Integer] -> probability: Float`.
        terminal: A list of terminal states.
        p_action: Local action probabilities as map
            `[state: Integer, action: Integer] -> probability: Float`
            as returned by `local_action_probabilities`.
        eps: The threshold to be used as convergence criterion. Convergence
            is assumed if the expected state visitation frequency changes
            less than the threshold on all states in a single iteration.

    Returns:
        The expected state visitation frequencies as map
        `[state: Integer] -> svf: Float`.
    """
    n_states, _, n_actions = p_transition.shape

    # 'fix' our transition probabilities to allow for convergence
    # we will _never_ leave any terminal state
    p_transition = np.copy(p_transition)
    p_transition[terminal, :, :] = 0.0

    # set-up transition matrices for each action
    p_transition = [np.array(p_transition[:, :, a]) for a in range(n_actions)]

    # actual forward-computation of state expectations
    d = np.zeros(n_states)

    delta = np.inf
    while delta > eps:
        d_ = [p_transition[a].T.dot(p_action[:, a] * d) for a in range(n_actions)]
        d_ = p_initial + np.array(d_).sum(axis=0)

        delta, d = np.max(np.abs(d_ - d)), d_

    return d


# -- plain maximum entropy (Ziebart et al. 2008) -------------------------------

def local_action_probabilities(p_transition, terminal, reward):
    """
    Compute the local action probabilities (policy) required for the edge
    frequency calculation for maximum entropy reinfocement learning.

    This is the backward pass of Algorithm 1 of the Maximum Entropy IRL
    paper by Ziebart et al. (2008).

    Args:
        p_transition: The transition probabilities of the MDP as table
            `[from: Integer, to: Integer, action: Integer] -> probability: Float`
            specifying the probability of a transition from state `from` to
            state `to` via action `action` to succeed.
        terminal: A set/list of terminal states.
        reward: The reward signal per state as table
            `[state: Integer] -> reward: Float`.

    Returns:
        The local action probabilities (policy) as map
        `[state: Integer, action: Integer] -> probability: Float`
    """
    n_states, _, n_actions = p_transition.shape

    er = np.exp(reward)
    p = [np.array(p_transition[:, :, a]) for a in range(n_actions)]

    # initialize at terminal states
    zs = np.zeros(n_states)
    zs[terminal] = 1.0

    # perform backward pass
    # This does not converge, instead we iterate a fixed number of steps. The
    # number of steps is chosen to reflect the maximum steps required to
    # guarantee propagation from any state to any other state and back in an
    # arbitrary MDP defined by p_transition.
    for _ in range(2 * n_states):
        za = np.array([er * p[a].dot(zs) for a in range(n_actions)]).T
        zs = za.sum(axis=1)

    # compute local action probabilities
    return za / zs[:, None]


def compute_expected_svf(p_transition, p_initial, terminal, reward, eps=1e-5):
    """
    Compute the expected state visitation frequency for maximum entropy IRL.

    This is an implementation of Algorithm 1 of the Maximum Entropy IRL
    paper by Ziebart et al. (2008).

    This function combines the backward pass implemented in
    `local_action_probabilities` with the forward pass implemented in
    `expected_svf_from_policy`.

    Args:
        p_transition: The transition probabilities of the MDP as table
            `[from: Integer, to: Integer, action: Integer] -> probability: Float`
            specifying the probability of a transition from state `from` to
            state `to` via action `action` to succeed.
        p_initial: The probability of a state being an initial state as map
            `[state: Integer] -> probability: Float`.
        terminal: A list of terminal states.
        reward: The reward signal per state as table
            `[state: Integer] -> reward: Float`.
        eps: The threshold to be used as convergence criterion for the
            expected state-visitation frequency. Convergence is assumed if
            the expected state visitation frequency changes less than the
            threshold on all states in a single iteration.

    Returns:
        The expected state visitation frequencies as map
        `[state: Integer] -> svf: Float`.
    """
    p_action = local_action_probabilities(p_transition, terminal, reward)
    return expected_svf_from_policy(p_transition, p_initial, terminal, p_action, eps)


def irl(p_transition, features, terminal, trajectories, optim, init, eps=1e-4, eps_esvf=1e-5):
    """
    Compute the reward signal given the demonstration trajectories using the
    maximum entropy inverse reinforcement learning algorithm proposed in the
    corresponding paper by Ziebart et al. (2008).

    Args:
        p_transition: The transition probabilities of the MDP as table
            `[from: Integer, to: Integer, action: Integer] -> probability: Float`
            specifying the probability of a transition from state `from` to
            state `to` via action `action` to succeed.
        features: The feature-matrix (e.g. as numpy array), mapping states
            to features, i.e. a matrix of shape (n_states x n_features).
        terminal: A list of terminal states.
        trajectories: A list of `Trajectory` instances representing the
            expert demonstrations.
        optim: The `Optimizer` instance to use for gradient-based
            optimization.
        init: The `Initializer` to use for initialization of the reward
            function parameters.
        eps: The threshold to be used as convergence criterion for the
            reward parameters. Convergence is assumed if all changes in the
            scalar parameters are less than the threshold in a single
            iteration.
        eps_svf: The threshold to be used as convergence criterion for the
            expected state-visitation frequency. Convergence is assumed if
            the expected state visitation frequency changes less than the
            threshold on all states in a single iteration.

    Returns:
        The reward per state as table `[state: Integer] -> reward: Float`.
    """
    n_states, _, n_actions = p_transition.shape
    _, n_features = features.shape

    # compute static properties from trajectories
    e_features = feature_expectation_from_trajectories(features, trajectories)
    p_initial = initial_probabilities_from_trajectories(n_states, trajectories)

    # basic gradient descent
    theta = init(n_features)
    delta = np.inf

    optim.reset(theta)
    while delta > eps:
        theta_old = theta.copy()

        # compute per-state reward
        reward = features.dot(theta)

        # compute the gradient
        e_svf = compute_expected_svf(p_transition, p_initial, terminal, reward, eps_esvf)
        grad = e_features - features.T.dot(e_svf)

        # perform optimization step and compute delta for convergence
        optim.step(grad)
        delta = np.max(np.abs(theta_old - theta))

    # re-compute per-state reward and return
    return features.dot(theta)


# -- maximum causal entropy (Ziebart 2010) -------------------------------------

def softmax(x1, x2):
    """
    Computes a soft maximum of both arguments.

    In case `x1` and `x2` are arrays, computes the element-wise softmax.

    Args:
        x1: Scalar or ndarray.
        x2: Scalar or ndarray.

    Returns:
        The soft maximum of the given arguments, either scalar or ndarray,
        depending on the input.
    """
    x_max = np.maximum(x1, x2)
    x_min = np.minimum(x1, x2)
    return x_max + np.log(1.0 + np.exp(x_min - x_max))


def local_causal_action_probabilities(p_transition, terminal, reward, discount, eps=1e-5):
    """
    Compute the local action probabilities (policy) required for the edge
    frequency calculation for maximum causal entropy reinfocement learning.

    This is Algorithm 9.1 from Ziebart's thesis (2010) combined with
    discounting for convergence reasons as proposed in the same thesis.

    Args:
        p_transition: The transition probabilities of the MDP as table
            `[from: Integer, to: Integer, action: Integer] -> probability: Float`
            specifying the probability of a transition from state `from` to
            state `to` via action `action` to succeed.
        terminal: Either the terminal reward function or a collection of
            terminal states. Iff `len(terminal)` is equal to the number of
            states, it is assumed to contain the terminal reward function
            (phi) as specified in Ziebart's thesis. Otherwise `terminal` is
            assumed to be a collection of terminal states from which the
            terminal reward function will be derived.
        reward: The reward signal per state as table
            `[state: Integer] -> reward: Float`.
        discount: A discounting factor as Float.
        eps: The threshold to be used as convergence criterion for the state
            partition function. Convergence is assumed if the state
            partition function changes less than the threshold on all states
            in a single iteration.

    Returns:
        The local action probabilities (policy) as map
        `[state: Integer, action: Integer] -> probability: Float`
    """
    n_states, _, n_actions = p_transition.shape

    # set up terminal reward function
    if len(terminal) == n_states:
        reward_terminal = np.array(terminal, dtype=np.float)
    else:
        reward_terminal = -np.inf * np.ones(n_states)
        reward_terminal[terminal] = 0.0

    # set up transition probability matrices
    p = [np.array(p_transition[:, :, a]) for a in range(n_actions)]

    # compute state log partition V and state-action log partition Q
    v = -1e200 * np.ones(n_states)  # np.dot doesn't behave with -np.inf

    delta = np.inf
    while delta > eps:
        v_old = v

        q = np.array([reward + discount * p[a].dot(v_old) for a in range(n_actions)]).T

        v = reward_terminal
        for a in range(n_actions):
            v = softmax(v, q[:, a])

        # for some reason numpy chooses an array of objects after reduction, force floats here
        v = np.array(v, dtype=float)

        delta = np.max(np.abs(v - v_old))

    # compute and return policy
    return np.exp(q - v[:, None])


def compute_expected_causal_svf(p_transition, p_initial, terminal, reward, discount,
                                eps_lap=1e-5, eps_svf=1e-5):
    """
    Compute the expected state visitation frequency for maximum causal
    entropy IRL.

    This is a combination of Algorithm 9.1 and 9.3 of Ziebart's thesis
    (2010). See `local_causal_action_probabilities` and
    `expected_svf_from_policy` for more details.

    Args:
        p_transition: The transition probabilities of the MDP as table
            `[from: Integer, to: Integer, action: Integer] -> probability: Float`
            specifying the probability of a transition from state `from` to
            state `to` via action `action` to succeed.
        p_initial: The probability of a state being an initial state as map
            `[state: Integer] -> probability: Float`.
        terminal: Either the terminal reward function or a collection of
            terminal states. Iff `len(terminal)` is equal to the number of
            states, it is assumed to contain the terminal reward function
            (phi) as specified in Ziebart's thesis. Otherwise `terminal` is
            assumed to be a collection of terminal states from which the
            terminal reward function will be derived.
        reward: The reward signal per state as table
            `[state: Integer] -> reward: Float`.
        discount: A discounting factor as Float.
        eps_lap: The threshold to be used as convergence criterion for the
            state partition function. Convergence is assumed if the state
            partition function changes less than the threshold on all states
            in a single iteration.
        eps_svf: The threshold to be used as convergence criterion for the
            expected state-visitation frequency. Convergence is assumed if
            the expected state visitation frequency changes less than the
            threshold on all states in a single iteration.
    """
    p_action = local_causal_action_probabilities(p_transition, terminal, reward, discount, eps_lap)
    return expected_svf_from_policy(p_transition, p_initial, terminal, p_action, eps_svf)


def irl_causal(X_df, p_transition, features, terminal, trajectories, optim, init, discount,eps=1e-4, eps_svf=1e-5, eps_lap=1e-5):
    """
    Compute the reward signal given the demonstration trajectories using the
    maximum causal entropy inverse reinforcement learning algorithm proposed
    Ziebart's thesis (2010).

    Args:
        p_transition: The transition probabilities of the MDP as table
            `[from: Integer, to: Integer, action: Integer] -> probability: Float`
            specifying the probability of a transition from state `from` to
            state `to` via action `action` to succeed.
        features: The feature-matrix (e.g. as numpy array), mapping states
            to features, i.e. a matrix of shape (n_states x n_features).
        terminal: Either the terminal reward function or a collection of
            terminal states. Iff `len(terminal)` is equal to the number of
            states, it is assumed to contain the terminal reward function
            (phi) as specified in Ziebart's thesis. Otherwise `terminal` is
            assumed to be a collection of terminal states from which the
            terminal reward function will be derived.
        trajectories: A list of `Trajectory` instances representing the
            expert demonstrations.
        optim: The `Optimizer` instance to use for gradient-based
            optimization.
        init: The `Initializer` to use for initialization of the reward
            function parameters.
        discount: A discounting factor for the log partition functions as
            Float.
        eps: The threshold to be used as convergence criterion for the
            reward parameters. Convergence is assumed if all changes in the
            scalar parameters are less than the threshold in a single
            iteration.
        eps_lap: The threshold to be used as convergence criterion for the
            state partition function. Convergence is assumed if the state
            partition function changes less than the threshold on all states
            in a single iteration.
        eps_svf: The threshold to be used as convergence criterion for the
            expected state-visitation frequency. Convergence is assumed if
            the expected state visitation frequency changes less than the
            threshold on all states in a single iteration.
    """
    n_states, _, n_actions = p_transition.shape
    _, n_features = features.shape

    # compute static properties from trajectories
    e_features = feature_expectation_from_trajectories(features, trajectories)
    p_initial = initial_probabilities_from_trajectories(n_states, trajectories)

    # basic gradient descent
    theta = init(n_features)
    delta = np.inf

    # input data
    epochs_infos = []

    optim.reset(theta)
    while delta > eps:
        theta_old = theta.copy()

        # compute per-state reward
        reward = features.dot(theta)

        epochs_infos.append(reward.tolist())

        # compute the gradient
        e_svf = compute_expected_causal_svf(p_transition, p_initial, terminal, reward, discount,
                                            eps_lap, eps_svf)

        grad = e_features - features.T.dot(e_svf)

        # perform optimization step and compute delta for convergence
        optim.step(grad)
        delta = np.max(np.abs(theta_old - theta))

    # re-compute per-state reward and return
    final_reward =features.dot(theta)
    epochs_infos.append(final_reward.tolist())

    return epochs_infos, final_reward


In [ ]:
import inspect
inspect.signature(irl_causal)

<Signature (X_df, p_transition, features, terminal, trajectories, optim, init, discount, eps=0.0001, eps_svf=1e-05, eps_lap=1e-05)>

In [ ]:
def update(reward_list, X_df):

    rankings = list(np.argsort(reward_list))
    # lowest to highest
    all_cluster_info = {}
    for i in range(len(reward_list)):
        # reward_maxent_causal[i] is the reward value
        cluster_id = i+1
        reward_value = reward_list[i]
        data = X_df[(X_df['cluster'] == i+1)]
        all_cluster_info[cluster_id] = data.mean().to_dict()
        all_cluster_info[cluster_id]['reward_value'] = reward_value
        # note: rankings are 0-indexed with the unhealthiest first (lowest)
        all_cluster_info[cluster_id]['ranking'] = rankings.index(i)
    # add to history
    return all_cluster_info



In [ ]:
import json
def extract_values(discount_val, features, cluster_algo, smooth_val, learning_rate=0.2, data=''):
  # choose our parameter initialization strategy:
  #   initialize parameters with constant
  init = O.Constant(1.0)

  # choose our optimization strategy:
  #   we select exponentiated stochastic gradient descent with linear learning-rate decay
  optim = O.ExpSga(lr=O.linear_decay(lr0=learning_rate))

  # actually do some inverse reinforcement learning
  # reward_maxent = maxent_irl(p_transition, features, terminal_states, trajectories, optim, init, eps= 1e-3)

  # send in the X_df dataframe
  epochs_infos, final_reward = irl_causal(data_non_normalized_df, p_transition, features, terminal_states, trajectories, optim, init, discount_val, eps=1e-3, eps_svf=1e-4, eps_lap=1e-4)

  if data != '':
    file_name = data+'mce_' + cluster_algo + str(discount_val)+'_smooth'+str(smooth_val)+'_lr'+str(learning_rate)+'.json'
  else:
    file_name = 'mce_' + cluster_algo + str(discount_val)+'_smooth'+str(smooth_val)+'.json'

  final_cluster_info = update(final_reward, data_non_normalized_df)
  print("----")
  # # print(final_cluster_info)
  # print("----")
  # print(epochs_infos)
  print("---")
  print(np.array(epochs_infos).shape)
  with open(file_name, 'w') as m:
    json.dump(epochs_infos, m)

  return epochs_infos

In [ ]:
# kmeans
# set up features: we use one feature vector per state (1 hot encoding for each cluster/state)
features = state_encoder.transform(np.arange(num_clusters).reshape(-1, 1))

epoch_reward_values = extract_values(discount_val, features, cluster_name, smoothing_value)



<ipython-input-581-954b00f430b7>:314: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  reward_terminal = np.array(terminal, dtype=np.float)


----
---
(9382, 100)


In [ ]:
print(epoch_reward_values[-1])

[5.64582000801477, 1.7153929821668132, 13.454589165675147, 2.8524446025378234, 2.8702467018548443, 3.0088296625437585, 7.229300678128408, 1.4191342929952517, 3.1592720984820875, 1.6639413393561924, 4.699903079741017, 2.8125622817576925, 1.8708954900536272, 6.345945442132183, 1.5462608535146878, 2.0364894357316525, 4.8866586595394645, 6.955292476693595, 7.825486302184945, 2.0478575683726725, 4.317737829274593, 1.9702387509949664, 10.262708509092482, 8.295162208471874, 5.430051225602344, 5.249962482084544, 3.5382785301235122, 2.9960486398315602, 1.6519974435328042, 2.58809712046632, 8.529288662604399, 4.363219070079707, 1.962512019655448, 5.36290261781417, 2.6567913263853318, 2.6205025417176078, 9.02937138661296, 4.40629274037059, 2.9589991744265585, 1.3409759907495193, 1.7777251700815924, 1.0568993939052131, 3.4513101268686475, 13.472223683159132, 1.3339686620539735, 1.319199430592301, 1.2198938463117301, 1.811217664103144, 1.1606625659564065, 1.410332412093611, 11.550482944378537, 13.3